In [1]:
import numpy as np
import os

In [2]:
spectrogram = os.listdir('spectrogram')
len(spectrogram)

17399

In [3]:
X, Y = [], []
for spec in spectrogram:
    Y.append(' '.join([i for i in os.path.splitext(spec)[0].split('-') if not i.isdigit()]))
    X.append(np.load('spectrogram/' + spec))

In [4]:
import tensorflow as tf
from tqdm import tqdm

In [5]:
chars = list(set([c for target in Y for c in target]))
num_classes = len(chars) + 2

idx2char = {idx + 1: char for idx, char in enumerate(chars)}
idx2char[0] = '<PAD>'
char2idx = {char: idx for idx, char in idx2char.items()}

Y = [[char2idx[c] for c in target] for target in Y]

In [6]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

def sparse_tuple_from(sequences, dtype=np.int32):
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

In [7]:
def pad_second_dim(x, desired_size):
    padding = tf.tile([[0]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1]], 0))
    return tf.concat([x, padding], 1)

_BATCH_NORM_EPSILON = 1e-5
_BATCH_NORM_DECAY = 0.997
_CONV_FILTERS = 32

def batch_norm(inputs, training):
    return tf.layers.batch_normalization(
      inputs=inputs, momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON,
      fused=True, training=training)

def _conv_bn_layer(inputs, padding, filters, kernel_size, strides, layer_id,
                   training):
    inputs = tf.pad(
      inputs,
      [[0, 0], [padding[0], padding[0]], [padding[1], padding[1]], [0, 0]])
    inputs = tf.layers.conv2d(
      inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides,
      padding="valid", use_bias=False, activation=tf.nn.relu6,
      name="cnn_{}".format(layer_id))
    return batch_norm(inputs, training)

def _rnn_layer(inputs, rnn_cell, rnn_hidden_size, layer_id, is_batch_norm,
               is_bidirectional, training):
    if is_batch_norm:
        inputs = batch_norm(inputs, training)
    
    fw_cell = rnn_cell(num_units=rnn_hidden_size,
                     name="rnn_fw_{}".format(layer_id))
    bw_cell = rnn_cell(num_units=rnn_hidden_size,
                     name="rnn_bw_{}".format(layer_id))

    if is_bidirectional:
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=fw_cell, cell_bw=bw_cell, inputs=inputs, dtype=tf.float32,
        swap_memory=True)
        rnn_outputs = tf.concat(outputs, -1)
    else:
        rnn_outputs = tf.nn.dynamic_rnn(
        fw_cell, inputs, dtype=tf.float32, swap_memory=True)

    return rnn_outputs

class Model:
    def __init__(
        self,
        size_layers,
        learning_rate,
        num_features,
        dropout = 1.0,
    ):
        self.X = tf.placeholder(tf.float32, [None, None, num_features])
        self.label = tf.placeholder(tf.int32, [None, None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])
        self.training = tf.placeholder(tf.bool, None)
        self.Y = tf.sparse_placeholder(tf.int32)
        x = tf.expand_dims(self.X, -1)

        inputs = _conv_bn_layer(
            x, padding=(20, 5), filters=_CONV_FILTERS, kernel_size=(41, 11),
            strides=(2, 2), layer_id=1, training=self.training)
        
        inputs = _conv_bn_layer(
            inputs, padding=(10, 5), filters=_CONV_FILTERS, kernel_size=(21, 11),
            strides=(2, 1), layer_id=2, training=self.training)
        
        batch_size = tf.shape(inputs)[0]
        feat_size = inputs.get_shape().as_list()[2]
        inputs = tf.reshape(
            inputs,
            [batch_size, -1, feat_size * _CONV_FILTERS // 4])
        print(inputs)
        
        seq_lens = tf.count_nonzero(
            tf.reduce_sum(inputs, -1), 1, dtype = tf.int32
        ) + 30
        filled = tf.fill(tf.shape(seq_lens), tf.shape(inputs)[1])
        seq_lens = tf.where(seq_lens > tf.shape(inputs)[1], filled, seq_lens)
        
        rnn_cell = tf.nn.rnn_cell.GRUCell
        for layer_counter in range(5):
            is_batch_norm = (layer_counter != 0)
            inputs = _rnn_layer(
              inputs, rnn_cell, size_layers, layer_counter + 1,
              is_batch_norm, True, self.training)
        

        logits = tf.layers.dense(inputs, num_classes)
        self.logits = logits
        time_major = tf.transpose(logits, [1, 0, 2])
        decoded, log_prob = tf.nn.ctc_beam_search_decoder(time_major, seq_lens)
        decoded = tf.to_int32(decoded[0])
        self.preds = tf.sparse_tensor_to_dense(decoded)
        self.cost = tf.reduce_mean(
            tf.nn.ctc_loss(
                self.Y,
                time_major,
                seq_lens
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        
        preds = self.preds[:, :tf.reduce_max(self.Y_seq_len)]
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        preds = pad_second_dim(preds, tf.reduce_max(self.Y_seq_len))
        y_t = tf.cast(preds, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.label, masks)
        self.mask_label = mask_label
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X, Y)
del X

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

size_layers = 128
learning_rate = 1e-3
num_layers = 2
batch_size = 32
epoch = 50

model = Model(size_layers, learning_rate, train_X[0].shape[1])
sess.run(tf.global_variables_initializer())

Tensor("Reshape:0", shape=(?, ?, 1600), dtype=float32)


In [10]:
for e in range(epoch):
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    total_cost, total_accuracy = 0, 0
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, len(train_X))]
        batch_x = tf.keras.preprocessing.sequence.pad_sequences(batch_x, dtype = 'float32', padding = 'post')
        y = train_Y[i : min(i + batch_size, len(train_X))]
        batch_y = sparse_tuple_from(y)
        batch_label, batch_len = pad_sentence_batch(y, 0)
        _, cost, accuracy = sess.run(
            [model.optimizer, model.cost, model.accuracy],
            feed_dict = {model.X: batch_x, model.Y: batch_y, 
                         model.label: batch_label, model.Y_seq_len: batch_len,
                        model.training: True},
        )
        total_cost += cost
        total_accuracy += accuracy
        pbar.set_postfix(cost = cost, accuracy = accuracy)
    total_cost /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    print('epoch %d, training average cost %f, training average accuracy %f'%(e + 1, total_cost, total_accuracy))
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'testing minibatch loop')
    total_cost, total_accuracy = 0, 0
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, len(test_X))]
        batch_x = tf.keras.preprocessing.sequence.pad_sequences(batch_x, dtype = 'float32', padding = 'post')
        y = test_Y[i : min(i + batch_size, len(test_X))]
        batch_y = sparse_tuple_from(y)
        batch_label, batch_len = pad_sentence_batch(y, 0)
        _, cost, accuracy = sess.run(
            [model.optimizer, model.cost, model.accuracy],
            feed_dict = {model.X: batch_x, model.Y: batch_y, 
                         model.label: batch_label, model.Y_seq_len: batch_len,
                        model.training:False},
        )
        total_cost += cost
        total_accuracy += accuracy
        pbar.set_postfix(cost = cost, accuracy = accuracy)
        
    total_cost /= (len(test_X) / batch_size)
    total_accuracy /= (len(test_X) / batch_size)
    print('epoch %d, testing average cost %f, testing average accuracy %f'%(e + 1, total_cost, total_accuracy))

testing minibatch loop:   0%|          | 0/136 [00:00<?, ?it/s]

epoch 1, training average cost 36.752024, training average accuracy 0.456052


minibatch loop:   0%|          | 0/408 [00:00<?, ?it/s]

epoch 1, testing average cost 33.640314, testing average accuracy 0.507403


testing minibatch loop:   0%|          | 0/136 [00:00<?, ?it/s]

...

epoch 49, testing average cost 29.081414, testing average accuracy 0.527855


testing minibatch loop:   0%|          | 0/136 [00:00<?, ?it/s]

epoch 50, training average cost 29.715384, training average accuracy 0.517801


testing minibatch loop: 100%|██████████| 136/136 [01:59<00:00,  1.15it/s, accuracy=0.523, cost=32.9]

epoch 50, testing average cost 36.579830, testing average accuracy 0.483868


In [12]:
import random

random_index = random.randint(0, len(test_X) - 1)
batch_x = test_X[random_index : random_index + 1]
print(
    'real:',
    ''.join(
        [idx2char[no] for no in test_Y[random_index : random_index + 1][0]]
    ),
)
batch_y = sparse_tuple_from(test_Y[random_index : random_index + 1])
pred = sess.run(model.preds, feed_dict = {model.X: batch_x, model.training: False})[0]
print('predicted:', ''.join([idx2char[no] for no in pred]))

real: tolong sebut poulard
predicted: tolong sebut a
